# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
from api_keys import weather_api_key
from citipy import citipy

In [7]:
city_data = pd.read_csv("../output_data/cities.csv")
city_data.dropna()
city_data

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [8]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

609

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [10]:
# Set base url and units variables, and combine them to get the base url
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
base_url = f"{url}appid={weather_api_key}&units={units}&q="

# Create empty lists to store the data for each field
city_id_list = []
city_name_list = []
country_name_list = []
lng_list = []
lat_list = []
temp_list = []
humidity_list = []
clouds_list = []
wind_speed_list = []

# Loop through cities
for city in cities:
    
    # Get the JSON response from the OpenWeather API
    response_json = requests.get(base_url + city).json()
    
    # Start the error handling
    # Retrieve the items for each field and append them to the empty lists  
    try:
        
        city_id = response_json['id']
        city_id_list.append(city_id)
           
        city_name = response_json['name']
        city_name_list.append(city_name)
         
        country_name = response_json['sys']['country']
        country_name_list.append(country_name)

        lng = response_json['coord']['lon']
        lng_list.append(lng)

        lat = response_json['coord']['lat']
        lat_list.append(lat)

        temp = response_json['main']['temp']
        temp_list.append(temp)

        humidity = response_json['main']['humidity']
        humidity_list.append(humidity)

        clouds = response_json['clouds']['all']
        clouds_list.append(clouds)

        wind_speed = response_json['wind']['speed']
        wind_speed_list.append(wind_speed)
    
        print(f"City Name: {city}, City ID: {city_id}, Temperature:{temp},  Humidity:{humidity}, Clouds:{clouds}, Wind Speed:{wind_speed}")
        print(f"--------------------------------------------------------------------------------------")
    # Print error message and move on to the next city if there is an error
    except:
        
        print("That record does not exist, searching for next record...")
        print("----------------------------------------------------------------------------------------")
    
        

City Name: bredasdorp, City ID: 1015776, Temperature:40.32,  Humidity:81, Clouds:5, Wind Speed:2.3
--------------------------------------------------------------------------------------
City Name: tuktoyaktuk, City ID: 6170031, Temperature:48.2,  Humidity:81, Clouds:90, Wind Speed:13.8
--------------------------------------------------------------------------------------
City Name: taltal, City ID: 3870243, Temperature:57.36,  Humidity:83, Clouds:5, Wind Speed:3.06
--------------------------------------------------------------------------------------
City Name: butaritari, City ID: 2110227, Temperature:82.69,  Humidity:77, Clouds:63, Wind Speed:11.65
--------------------------------------------------------------------------------------
City Name: kapaa, City ID: 5848280, Temperature:81.63,  Humidity:77, Clouds:17, Wind Speed:3
--------------------------------------------------------------------------------------
City Name: yellowknife, City ID: 6185377, Temperature:69.89,  Humidity:43,

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
city_data_df = pd.DataFrame({
    "City" : city_id_list,
    "Lat" : lats_list,
    "Lng" : lngs_list,
    "Max Temp" : temp_max,
    "Humidity" : humidity_list,
    "Clouds" : cloud_list,
    "Wind Speed" : wind_speed_list,
    "Country" : country_list,
    "Date" : date_list    
})

NameError: name 'lats_list' is not defined

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression